In [10]:
import torch
import evaluate
import os
import csv
from tqdm import tqdm

def load_texts(file_path):
    """
    Load texts from a file and return as a list of stripped strings.
    """
    with open(file_path, 'r', encoding='utf-8') as f:
        return [line.strip() for line in f.readlines()]

def main():
    # Define explicit pairs of hypothesis and target files
    # Format: (hypothesis_file_path, target_file_path)
    file_pairs = [
        ("generated_hypotheses/ikatdata_checkpoint16.hypo", "D3test/ikatdata.target"),
        ("generated_hypotheses/ikatdata_checkpoint50.hypo", "D3test/ikatdata.target"),
        ("generated_hypotheses/jandata_checkpoint16.hypo", "D2test/jandata.target"),
        ("generated_hypotheses/jandata_checkpoint50.hypo", "D2test/jandata.target"),
        ("generated_hypotheses/semevaldata_checkpoint16.hypo", "D1test/semevaldata.target"),
        ("generated_hypotheses/semevaldata_checkpoint50.hypo", "D1test/semevaldata.target")
    ]
    
    # Output CSV file to store evaluation results
    output_csv = 'evaluation_results.csv'
    
    # Initialize metrics
    bleu = evaluate.load('bleu')
    bertscore = evaluate.load('bertscore')
    rouge = evaluate.load('rouge')  # Optional, if ROUGE is desired
    
    # Prepare CSV header
    csv_header = ['Model', 'Source File', 'BLEU-1', 'BLEU-2', 
                  'ROUGE-1', 'ROUGE-2', 'ROUGE-L', 
                  'BERTScore Precision', 'BERTScore Recall', 'BERTScore F1']
    
    # Open CSV file for writing
    with open(output_csv, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(csv_header)
        
        print(f"Starting evaluation of {len(file_pairs)} file pairs.\n")
        
        # Iterate over each file pair with a progress bar
        for hypo_path, target_path in tqdm(file_pairs, desc="Evaluating file pairs"):
            # Extract model identifier and source file name from the hypothesis file name
            hypo_filename = os.path.basename(hypo_path)
            try:
                sourcefile, modelid = hypo_filename.split('_checkpoint')
                modelid = f"checkpoint{modelid.split('.hypo')[0]}"
            except Exception as e:
                print(f"Filename format error for {hypo_filename}: {e}")
                continue  # Skip this pair if filename is not as expected
            
            # Check if both files exist
            if not os.path.exists(hypo_path):
                print(f"Hypothesis file not found: {hypo_path}. Skipping.")
                continue
            if not os.path.exists(target_path):
                print(f"Target file not found: {target_path}. Skipping.")
                continue
            
            # Load texts
            generated_hypo_texts = load_texts(hypo_path)
            target_texts = load_texts(target_path)
            
            # Validate that all datasets have the same number of samples
            if len(target_texts) != len(generated_hypo_texts):
                print(f"Line count mismatch between hypothesis and target for {hypo_filename}. Skipping.")
                continue  # Skip if line counts do not match
            
            # Compute BLEU-1
            bleu1 = bleu.compute(
                predictions=generated_hypo_texts,
                references=[[ref] for ref in target_texts],
                max_order=1
            )
            
            # Compute BLEU-2
            bleu2 = bleu.compute(
                predictions=generated_hypo_texts,
                references=[[ref] for ref in target_texts],
                max_order=2
            )
            
            # Compute ROUGE
            ro = rouge.compute(
                predictions=generated_hypo_texts,
                references=target_texts,
                use_stemmer=True
            )
            
            # Compute BERTScore
            bs = bertscore.compute(
                predictions=generated_hypo_texts,
                references=target_texts,
                lang='en',
                model_type='bert-base-uncased',
                verbose=False
            )
            
            # Calculate average BERTScore metrics
            bert_precision = sum(bs['precision']) / len(bs['precision']) * 100
            bert_recall = sum(bs['recall']) / len(bs['recall']) * 100
            bert_f1 = sum(bs['f1']) / len(bs['f1']) * 100
            
            # Write results to CSV
            writer.writerow([
                modelid,
                sourcefile,
                f"{bleu1['bleu'] * 100:.2f}",
                f"{bleu2['bleu'] * 100:.2f}",
                f"{ro['rouge1'] * 100:.2f}",
                f"{ro['rouge2'] * 100:.2f}",
                f"{ro['rougeL'] * 100:.2f}",
                f"{bert_precision:.2f}",
                f"{bert_recall:.2f}",
                f"{bert_f1:.2f}"
            ])
            
            # Print summary for this pair
            print(f"Evaluated {hypo_filename}: BLEU-1={bleu1['bleu'] * 100:.2f}, BLEU-2={bleu2['bleu'] * 100:.2f}, ROUGE-1={ro['rouge1'] * 100:.2f}, ROUGE-2={ro['rouge2'] * 100:.2f}, ROUGE-L={ro['rougeL'] * 100:.2f}, BERTScore F1={bert_f1:.2f}")
        
        print(f"\nEvaluation completed. Results saved to {output_csv}.")

# Execute the evaluation
main()


Starting evaluation of 6 file pairs.



Evaluating file pairs:  17%|█▋        | 1/6 [00:01<00:09,  1.89s/it]

Evaluated ikatdata_checkpoint16.hypo: BLEU-1=65.53, BLEU-2=58.81, ROUGE-1=66.05, ROUGE-2=53.27, ROUGE-L=63.53, BERTScore F1=75.85


Evaluating file pairs:  33%|███▎      | 2/6 [00:02<00:05,  1.28s/it]

Evaluated ikatdata_checkpoint50.hypo: BLEU-1=61.09, BLEU-2=51.69, ROUGE-1=62.66, ROUGE-2=45.59, ROUGE-L=57.59, BERTScore F1=70.99


...

Evaluated jandata_checkpoint50.hypo: BLEU-1=8.52, BLEU-2=4.26, ROUGE-1=27.58, ROUGE-2=8.57, ROUGE-L=19.97, BERTScore F1=46.89


Evaluating file pairs:  83%|████████▎ | 5/6 [00:11<00:02,  2.97s/it]

Evaluated semevaldata_checkpoint16.hypo: BLEU-1=57.01, BLEU-2=50.65, ROUGE-1=60.76, ROUGE-2=48.35, ROUGE-L=58.17, BERTScore F1=73.28


Evaluating file pairs: 100%|██████████| 6/6 [00:17<00:00,  2.85s/it]

Evaluated semevaldata_checkpoint50.hypo: BLEU-1=61.20, BLEU-2=53.28, ROUGE-1=60.61, ROUGE-2=46.57, ROUGE-L=56.91, BERTScore F1=71.48

Evaluation completed. Results saved to evaluation_results.csv.
